<a href="https://colab.research.google.com/github/JMacnar/ring_analysis/blob/master/sixmembered_rings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install dependencies
%%bash -s 
if [ ! -f Py3Dmol_READY ]; then
  pip install py3Dmol
  touch Py3Dmol_READY
fi

if [ ! -f GIT_READY ]; then
  # install dependencies
  git clone https://github.com/JMacnar/ring_analysis.git
  cp ring_analysis/7ol5.pdb .
  touch GIT_READY
fi
# setup conda
if [ ! -f CONDA_READY ]; then
  wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.12.0-Linux-x86_64.sh
  chmod +x Miniconda3-py37_4.12.0-Linux-x86_64.sh
  bash ./Miniconda3-py37_4.12.0-Linux-x86_64.sh -b -f -p /usr/local/ > /dev/null
  conda install -y -q -c conda-forge py3dmol numpy > /dev/null
  touch CONDA_READY
fi


In [ ]:
#@title Upload PDB file and provide a ligand's three-letter PDB code

from google.colab import files
try:
  uploaded = files.upload()
  pdb_name = next(iter(uploaded))
except:
   pdb_name = "7ol5.pdb"
ligand_code = "EPE" #@param {type:"string"}
#@markdown The distance around a ligand used to visualize the structure of the ligand and its surroundings also can be provided
distance = 5 #@param {type:"integer"}


In [ ]:
#@title Run ring analysis
!python ring_analysis/sixmembered_ring.py {pdb_name} {ligand_code} -d {distance}

In [17]:
#@title Display 3D structure {run: "auto"}
import glob
import py3Dmol
color = "rainbow" #@param ["B-factor", "rainbow"]
show_sidechains = False #@param {type:"boolean"}
show_mainchains = False #@param {type:"boolean"}

pdb_file = glob.glob(ligand_code+'-*-?-*.pdb')

def show_pdb(show_sidechains=False, show_mainchains=False, color="rainbow"):
  view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
  view.addModel(open(pdb_file[0],'r').read(),'pdb')

  if color == "rainbow":
    view.setStyle({'cartoon': {'color':'spectrum'}})
  elif color == "B-factor":
    view.setStyle({'cartoon': {'colorscheme':{'prop':'b','gradient':'sinebow','min':0,'max':70}}})
  if show_sidechains:
    BB = ['C','O','N']
    view.addStyle({'and':[{'resn':["GLY","PRO"],'invert':True},{'atom':BB,'invert':True}]},
                        {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"GLY"},{'atom':'CA'}]},
                        {'sphere':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"PRO"},{'atom':['C','O'],'invert':True}]},
                        {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})  
  if show_mainchains:
    BB = ['C','O','N','CA']
    view.addStyle({'atom':BB},{'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})

  view.zoomTo()
  return view


show_pdb(show_sidechains, show_mainchains, color).show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol